# 01-polars-vs-pandas

The objective of this notebook is to evaluate and demonstrate the utility of [polars](https://github.com/pola-rs/polars), particularly in comparison against the more traditional pandas.

# 1. Env

In [1]:
import pandas as pd
import polars as pl
import numpy as np
from google.cloud import bigquery
import pandas_gbq as pd_gbq
import os

# 2. DataFrame Creation

In [2]:
length = 10000000
a = np.random.random(length)
b = np.random.randint(0, 5, size=length)
c = np.random.randint(0, 10, size=length)

In [3]:
fruit_list = ["banana", "apple", "cherry", "pineapple", "blueberry"]
fruit_weights = 5 + 4 * np.random.random(len(fruit_list))
fruits = np.random.choice(fruit_list, length)

In [4]:
%%time
pd_df = pd.DataFrame(
    {
        "a": a,
        "b": b,
        "c": c,
        "fruit": fruits
    }
)

pd_fruit_weights = pd.DataFrame(
    {
        "fruit": fruit_list,
        "fruit_weight": fruit_weights
    }
)

CPU times: user 1.31 s, sys: 200 ms, total: 1.51 s
Wall time: 1.51 s


In [5]:
%%time
pl_df = pl.DataFrame(
    {
        "a": a,
        "b": b,
        "c": c,
        "fruit": fruits
    }
)

pl_fruit_weights = pl.DataFrame(
    {
        "fruit": fruit_list,
        "fruit_weight": fruit_weights
    }
)

CPU times: user 3.73 s, sys: 729 ms, total: 4.46 s
Wall time: 4.48 s


# 3. Simple Aggregates

#### `sum`

In [6]:
%%time
pd_df[["a", "b", "c"]].sum()

CPU times: user 106 ms, sys: 41.8 ms, total: 148 ms
Wall time: 156 ms


a    5.000870e+06
b    2.000412e+07
c    4.500250e+07
dtype: float64

In [7]:
%%time
pl_df[["a", "b", "c"]].sum()

CPU times: user 42.3 ms, sys: 2.71 ms, total: 45 ms
Wall time: 21.4 ms


a,b,c
f64,i64,i64
5.000870038898128e6,20004125,45002495


#### `mean`

In [8]:
%%time
pd_df[["a", "b", "c"]].mean()

CPU times: user 83.1 ms, sys: 11.7 ms, total: 94.8 ms
Wall time: 93.4 ms


a    0.500087
b    2.000412
c    4.500249
dtype: float64

In [9]:
%%time
pl_df[["a", "b", "c"]].mean()

CPU times: user 17.2 ms, sys: 987 µs, total: 18.2 ms
Wall time: 9.41 ms


a,b,c
f64,f64,f64
0.5000870038898128,2.0004125,4.5002495


# 4. Groupby

#### one aggregate

In [10]:
%%time
(pd_df
 .groupby("fruit")
 ["a"]
 .mean())

CPU times: user 487 ms, sys: 35.8 ms, total: 523 ms
Wall time: 527 ms


fruit
apple        0.500170
banana       0.499984
blueberry    0.500221
cherry       0.500148
pineapple    0.499913
Name: a, dtype: float64

In [11]:
%%time
(pl_df
 .groupby("fruit")
 .agg(pl.mean("a"))
)

CPU times: user 537 ms, sys: 150 ms, total: 687 ms
Wall time: 127 ms


fruit,a_mean
str,f64
"""cherry""",0.5001478698638339
"""apple""",0.5001701836292793
"""blueberry""",0.5002207580751732
"""banana""",0.49998364192450545
"""pineapple""",0.49991268218246254


#### multiple aggregates

In [12]:
%%time
(pd_df
 .groupby("fruit")
 .mean())

CPU times: user 638 ms, sys: 99.1 ms, total: 737 ms
Wall time: 743 ms


,a,b,c
fruit,,,
apple,0.500170,1.999868,4.501321
banana,0.499984,2.001210,4.499238
blueberry,0.500221,1.999571,4.496202
cherry,0.500148,2.000871,4.502040
pineapple,0.499913,2.000543,4.502445


In [13]:
%%time
(pl_df
 .groupby("fruit")
 .agg(pl.mean("*"))
)

CPU times: user 608 ms, sys: 81.3 ms, total: 689 ms
Wall time: 112 ms


fruit,a_mean,b_mean,c_mean
str,f64,f64,f64
"""apple""",0.5001701836292793,1.9998675417905651,4.501320833277101
"""blueberry""",0.5002207580751732,1.9995713904902703,4.496201779554676
"""pineapple""",0.49991268218246254,2.000543239516652,4.502445077585298
"""banana""",0.49998364192450545,2.001209559873214,4.499238212291508
"""cherry""",0.5001478698638339,2.0008709267541094,4.502039999599804


# 5. Join

In [14]:
%%time
(pd_df
 .merge(pd_fruit_weights, how="left", on="fruit")
)

CPU times: user 1.24 s, sys: 101 ms, total: 1.34 s
Wall time: 1.35 s


,a,b,c,fruit,fruit_weight
0,0.423895,3,8,banana,7.510517
1,0.704444,0,1,cherry,7.443748
2,0.646311,0,9,cherry,7.443748
3,0.071881,0,6,apple,6.136765
4,0.513362,3,3,blueberry,7.247517
...,...,...,...,...,...
9999995,0.572503,0,1,pineapple,7.653308
9999996,0.645806,2,3,blueberry,7.247517
9999997,0.335286,1,2,cherry,7.443748
9999998,0.966750,3,3,pineapple,7.653308


In [15]:
%%time
(pl_df
 .join(pl_fruit_weights, on="fruit")
)

CPU times: user 1.05 s, sys: 232 ms, total: 1.28 s
Wall time: 372 ms


a,b,c,fruit,fruit_weight
f64,i64,i64,str,f64
0.42389526832474755,3,8,"""banana""",7.510516742997247
0.7044439624406613,0,1,"""cherry""",7.443748002301069
0.6463109702129014,0,9,"""cherry""",7.443748002301069
0.07188138038575087,0,6,"""apple""",6.136765366488852
0.513362095547555,3,3,"""blueberry""",7.2475166750944275
0.34407538691907635,4,6,"""blueberry""",7.2475166750944275
0.38457867586316585,1,5,"""banana""",7.510516742997247
0.17206711348006065,0,8,"""apple""",6.136765366488852
0.09314081281100628,4,1,"""cherry""",7.443748002301069


# 6. I/O

#### `to_csv`

In [16]:
%%time
pd_df.to_csv('./pd_df.csv')

CPU times: user 21.2 s, sys: 609 ms, total: 21.8 s
Wall time: 21.9 s


In [17]:
%%time
pl_df.to_csv('./pl_df.csv')

CPU times: user 1.42 s, sys: 377 ms, total: 1.8 s
Wall time: 1.83 s


#### `from_csv`

In [18]:
%%time
_ = pd.read_csv('./pd_df.csv')

CPU times: user 2.53 s, sys: 371 ms, total: 2.9 s
Wall time: 2.98 s


In [19]:
%%time
_ = pl.read_csv('./pl_df.csv')

CPU times: user 2.89 s, sys: 422 ms, total: 3.32 s
Wall time: 821 ms


#### `to_parquet`

In [20]:
%%time
pd_df.to_parquet('./pd_df.parquet')

CPU times: user 1.17 s, sys: 122 ms, total: 1.29 s
Wall time: 1.34 s


In [21]:
%%time
pl_df.to_parquet('./pl_df.parquet')

CPU times: user 835 ms, sys: 35.5 ms, total: 870 ms
Wall time: 1.08 s


#### `from_parquet`

In [22]:
%%time
_ = pd.read_parquet('./pd_df.parquet')

CPU times: user 903 ms, sys: 392 ms, total: 1.3 s
Wall time: 887 ms


In [23]:
%%time
_ = pl.read_parquet('./pl_df.parquet')

CPU times: user 764 ms, sys: 243 ms, total: 1.01 s
Wall time: 865 ms


#### Reading from BigQuery

In [24]:
QUERY = '''
SELECT
  *
FROM
  `dhub-location-production.address_science.address_profiled_orders`
WHERE
  order_created_date_utc >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY)
'''
client = bigquery.Client()

In [25]:
%%time
# https://cloud.google.com/bigquery/docs/pandas-gbq-migration
pd_bigquery_df = client.query(QUERY).to_dataframe()

CPU times: user 61.8 ms, sys: 26.8 ms, total: 88.7 ms
Wall time: 3.06 s


In [26]:
display(type(pd_bigquery_df))
pd_bigquery_df.head()

pandas.core.frame.DataFrame

,order_uuid,global_entity_id,address_uuid,pd_customer_id,is_own_delivery,is_delivery,is_gross_order,is_pickup,is_cancelled,is_address_cancelled,...,customer_count_past_address_cancellations,customer_count_address_cancellations,customer_seconds_since_last_order,customer_median_dropoff_error,customer_median_at_customer_time,hour_of_day,day_of_week,order_created_at_local,order_created_at_utc,order_created_date_utc


In [27]:
%%time
# https://pola-rs.github.io/polars-book/user-guide/howcani/io/google-big-query.html
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

pl_bigquery_df = pl.from_arrow(rows.to_arrow())

/Users/b.feifke/miniconda3/envs/location-data-science/lib/python3.9/site-packages/polars/utils.py:23: UserWarning: Conversion of (potentially) timezone aware to naive datetimes. TZ information may be lost
  warnings.warn(


RuntimeError: Other("empty table")

In [28]:
display(type(pl_bigquery_df))
pl_bigquery_df.head()

NameError: name 'pl_bigquery_df' is not defined

#### clean up directory of I/O files

In [29]:
for file in ["./pd_df.csv", "./pl_df.csv", "./pd_df.parquet", "./pl_df.parquet"]:
    os.remove(file)

# 7. String Methods

#### slice

In [ ]:
%%time
pd_df["fruit"].str[1:5] # The arguments are "start" and "stop" index.

In [ ]:
%%time
pl_df["fruit"].str.slice(1, 4) # The arguments are "start" index and "length"

# 7. Indexing

#### Setting new values to some rows of a column of a dataframe

##### pandas

In [52]:
banana_mask = pd_df["fruit"] == "banana"

In [56]:
pd_df[banana_mask]["c"] * 40

0          320
6          200
12         240
17         280
23         200
          ... 
9999975      0
9999978     80
9999983    280
9999984      0
9999991    240
Name: c, Length: 1999901, dtype: int64

In [60]:
pd_df.loc[banana_mask, "a"] = (pd_df[banana_mask]["c"] * 40).to_numpy()

In [62]:
(pd_df[banana_mask]["c"] * 40).to_numpy()

array([320, 200, 240, ..., 280,   0, 240])

In [61]:
pd_df.head()loc[banana_mask]

,a,b,c,fruit
0,320.0,3,8,banana
6,200.0,1,5,banana
12,240.0,2,6,banana
17,280.0,0,7,banana
23,200.0,0,5,banana
...,...,...,...,...
9999975,0.0,2,0,banana
9999978,80.0,1,2,banana
9999983,280.0,2,7,banana
9999984,0.0,4,0,banana


##### polars

In [63]:
banana_mask = pl_df["fruit"] == "banana"

In [64]:
# banana pi >:O
pl_df[banana_mask, "a"] = 3.14159

In [67]:
# pl_df[banana_mask, "a"] = pl_df["c"][banana_mask].to_numpy() > 8
pl_df[pl.col("a")].filter(banana_mask)# = pl_df.filter(banana_mask)[pl.col("c")] * 40

AttributeError: 'Series' object has no attribute '_pyexpr'

In [51]:
pl_df[banana_mask]

a,b,c,fruit
f64,i64,i64,str
3.14159,3,8,"""banana"""
3.14159,1,5,"""banana"""
3.14159,2,6,"""banana"""
3.14159,0,7,"""banana"""
3.14159,0,5,"""banana"""
3.14159,0,4,"""banana"""
3.14159,0,9,"""banana"""
3.14159,2,4,"""banana"""
3.14159,0,7,"""banana"""


# 8. Conclusion

Polars supports all the same basic functionality as Pandas, including easy interaction with cloud providers, but at least 4 times faster for almost all basic functions. Furthermore, Polars and Pandas are commensurately expressive, with some differences (e.g. Polars's dataframe output displays are more explicitly clear regarding data-types).